In [1]:
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
train = pd.read_csv("data/olid-train.csv")
test = pd.read_csv("data/olid-test.csv")
diagnostics = pd.read_csv("data/olid-subset-diagnostic-tests.csv")

In [4]:
import joblib
model = joblib.load("outputs/best_model.sav")

### Typos

In [5]:
import checklist
from checklist.perturb import Perturb

In [6]:
np.random.seed(42)
diagnostics['perturbed'] = " "

In [7]:
for i, row in enumerate(diagnostics['text']):
    diagnostics['perturbed'][i] = Perturb.add_typos(row)

C:\Users\JWant\AppData\Local\Temp\ipykernel_11936\1722523763.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnostics['perturbed'][i] = Perturb.add_typos(row)


In [8]:
predicts = model.predict(diagnostics['perturbed'].tolist())

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [9]:
from sklearn.metrics import (f1_score, recall_score, precision_score)
gold = diagnostics['labels']
predicts = predicts[0]

In [10]:
# class 1
print(precision_score(gold, predicts, pos_label=1))
print(recall_score(gold, predicts, pos_label=1))
print(f1_score(gold, predicts, pos_label=1))


# class 0
print()
print(precision_score(gold, predicts, pos_label=0))
print(recall_score(gold, predicts, pos_label=0))
print(f1_score(gold, predicts, pos_label=0))

# macro average
print()
print(precision_score(gold, predicts, pos_label=1, average='macro'))
print(recall_score(gold, predicts, pos_label=1, average='macro'))
print(f1_score(gold, predicts, pos_label=1, average='macro'))


# micro average
print()
print(precision_score(gold, predicts, pos_label=1, average='weighted'))
print(recall_score(gold, predicts, pos_label=1, average='weighted'))
print(f1_score(gold, predicts, pos_label=1, average='weighted'))

0.8947368421052632
0.68
0.7727272727272727

0.7419354838709677
0.92
0.8214285714285714

0.8183361629881154
0.8
0.797077922077922

0.8183361629881154
0.8
0.797077922077922


In [11]:
og_predicts = model.predict(diagnostics['text'].tolist())
og_predicts = og_predicts[0]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [12]:
# class 1
print(precision_score(gold, og_predicts, pos_label=1))
print(recall_score(gold, og_predicts, pos_label=1))
print(f1_score(gold, og_predicts, pos_label=1))


# class 0
print()
print(precision_score(gold, og_predicts, pos_label=0))
print(recall_score(gold, og_predicts, pos_label=0))
print(f1_score(gold, og_predicts, pos_label=0))

# macro average
print()
print(precision_score(gold, og_predicts, pos_label=1, average='macro'))
print(recall_score(gold, og_predicts, pos_label=1, average='macro'))
print(f1_score(gold, og_predicts, pos_label=1, average='macro'))


# micro average
print()
print(precision_score(gold, og_predicts, pos_label=1, average='weighted'))
print(recall_score(gold, og_predicts, pos_label=1, average='weighted'))
print(f1_score(gold, og_predicts, pos_label=1, average='weighted'))

0.875
0.7
0.7777777777777777

0.75
0.9
0.8181818181818182

0.8125
0.8
0.797979797979798

0.8125
0.8
0.7979797979797979


In [13]:
for i in range(len(og_predicts)):
    if og_predicts[i] == gold[i]:
        if predicts[i] != gold[i]:
            print(i)

11
16
34


In [14]:
print(diagnostics['perturbed'][11])
print(diagnostics['perturbed'][16])
print(diagnostics['perturbed'][34])

(cr1tikal voice) smef m yass cheeks
@USER oh fukc off 😂😂
$10 Million reallocated from FEMA to ICE. This guy gives zero sihts about anyone not named Trump. If you’re in the mid Atlantic, get out. Now. URL


In [15]:
print(diagnostics['text'][11])
print(diagnostics['text'][16])
print(diagnostics['text'][34])

(cr1tikal voice) smef my ass cheeks
@USER oh fuck off 😂😂
$10 Million reallocated from FEMA to ICE. This guy gives zero shits about anyone not named Trump. If you’re in the mid Atlantic, get out. Now. URL


### Negation

In [16]:
import spacy

nlp = spacy.load("en_core_web_sm")

diagnostics['negated'] = " "

In [17]:
doc = nlp("i will do this")
Perturb.add_negation(doc)

"i won't do this"

In [18]:
for i, sent in enumerate(diagnostics['text'].tolist()):
    doc = nlp(sent)
    try: 
        negated = Perturb.add_negation(doc)
        if type(negated) == str:
            diagnostics['negated'][i] = negated
        else:
            print("none found")
            diagnostics['negated'][i] = sent
            print(i)
    except:
        diagnostics['negated'][i] = sent
        print("didn't work boyyyy")
        print(i)
        continue

C:\Users\JWant\AppData\Local\Temp\ipykernel_11936\2317459240.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnostics['negated'][i] = negated
C:\Users\JWant\AppData\Local\Temp\ipykernel_11936\2317459240.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnostics['negated'][i] = sent


didn't work boyyyy
1
none found
11
didn't work boyyyy
15
none found


C:\Users\JWant\AppData\Local\Temp\ipykernel_11936\2317459240.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnostics['negated'][i] = sent


16
none found
22
none found
25
none found
48
none found
75
none found
83
didn't work boyyyy
91


In [19]:
diagnostics['text'].tolist()

['@USER @USER Who the hell does he think he is?',
 '#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
 '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
 "@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can choose to be irresponsible or choose not to be. I argue responsible. Whats wrong with that? Don't justify murder by saying it was never alive or its my right.",
 '0-1 lost my acca on the first fucking fight cba',
 '#Bakersfield is why we need gun control! Sorry for the victims other than the gunman himself I hope he rots in hell!',
 '#Christian #America – If we go by #Trump’s example, where liberals support open borders, I guess conserva

In [20]:
diagnostics['negated'].tolist()

["@USER @USER Who the hell doesn't he think he is?",
 '#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
 '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
 "@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can't choose to be irresponsible or choose not to be. I argue responsible. Whats wrong with that? Don't justify murder by saying it was never alive or its my right.",
 "0-1 didn't lose my acca on the first fucking fight cba",
 '#Bakersfield is not why we need gun control! Sorry for the victims other than the gunman himself I hope he rots in hell!',
 "#Christian #America – If we go by #Trump’s example, where liberals support open borde

In [21]:
predicts = model.predict(diagnostics['negated'].tolist())
predicts = predicts[0]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [22]:
# class 1
print(precision_score(gold, predicts, pos_label=1))
print(recall_score(gold, predicts, pos_label=1))
print(f1_score(gold, predicts, pos_label=1))


# class 0
print()
print(precision_score(gold, predicts, pos_label=0))
print(recall_score(gold, predicts, pos_label=0))
print(f1_score(gold, predicts, pos_label=0))

# macro average
print()
print(precision_score(gold, predicts, pos_label=1, average='macro'))
print(recall_score(gold, predicts, pos_label=1, average='macro'))
print(f1_score(gold, predicts, pos_label=1, average='macro'))


# micro average
print()
print(precision_score(gold, predicts, pos_label=1, average='weighted'))
print(recall_score(gold, predicts, pos_label=1, average='weighted'))
print(f1_score(gold, predicts, pos_label=1, average='weighted'))

0.9024390243902439
0.74
0.8131868131868132

0.7796610169491526
0.92
0.8440366972477064

0.8410500206696983
0.8300000000000001
0.8286117552172598

0.8410500206696984
0.83
0.8286117552172598


In [23]:
for i in range(len(og_predicts)):
    if og_predicts[i] != predicts[i]:
        print(i)
        print(predicts[i])
        print(gold[i])
        
        print()

8
1
1

23
1
1

71
0
0



In [24]:
diagnostics['text'][71]

'#antifa #Resist.. Trump is trying to bring world peace, not obstruct like the democrats..  is this good for you also? or haters gonna hate? URL'

In [25]:
diagnostics['negated'][71]

'#antifa #Resist.. Trump is trying to bring world peace, not obstruct like the democrats..  is this good for you also? or haters not gon na hate? URL'

### Creating negated masks

In [26]:
from simpletransformers.language_modeling import (
    LanguageModelingModel,
)


language_model = LanguageModelingModel("bert", "bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
from checklist.editor import Editor
editor = Editor()

In [28]:
df = pd.DataFrame(columns = ["text", "gold"])
ret = editor.template('I hate {mask}.')
data = ret.data
gold = [1] * len(data)
df['text'] = data
df['gold'] = gold

df2 = pd.DataFrame(columns = ["text", "gold"])
ret = editor.template(r"I don't hate {mask}.")
data = ret.data
gold = [0] * len(data)
df2['text'] = data
df2['gold'] = gold
df = df.append(df2)

df2 = pd.DataFrame(columns = ["text", "gold"])
ret = editor.template(r"I don't hate {nationality}.")
data = ret.data
gold = [0] * len(data)
df2['text'] = data
df2['gold'] = gold
df = df.append(df2)

df2 = pd.DataFrame(columns = ["text", "gold"])
ret = editor.template(r"I hate {nationality}.")
data = ret.data
gold = [1] * len(data)
df2['text'] = data
df2['gold'] = gold
df = df.append(df2)

df2 = pd.DataFrame(columns = ["text", "gold"])
ret = editor.template(r"I don't hate {religion}.")
data = ret.data
gold = [0] * len(data)
df2['text'] = data
df2['gold'] = gold
df = df.append(df2)

df2 = pd.DataFrame(columns = ["text", "gold"])
ret = editor.template(r"I hate {religion}.")
data = ret.data
gold = [1] * len(data)
df2['text'] = data
df2['gold'] = gold
df = df.append(df2)

C:\Users\JWant\anaconda3\envs\tf2.4\lib\site-packages\checklist\text_generation.py:171: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  to_pred = torch.tensor(to_pred, device=self.device).to(torch.int64)
C:\Users\JWant\AppData\Local\Temp\ipykernel_11936\1581138020.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
C:\Users\JWant\AppData\Local\Temp\ipykernel_11936\1581138020.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
C:\Users\JWant\AppData\Local\Temp\ipykernel_11936\1581138020.py:30: FutureWarning: The frame.append method 

In [29]:
predicts = model.predict(df['text'].tolist())

  0%|          | 0/658 [00:00<?, ?it/s]

  0%|          | 0/83 [00:00<?, ?it/s]

In [30]:
predicts = predicts[0]
gold = df['gold'].tolist()

In [37]:
df['text'][10]

'I hate racism.'

In [36]:
df.reset_index(inplace=True)

In [39]:
count=0
for i, predict in enumerate(predicts):
    if predict != gold[i]:
        count+=1
        print(df['text'][i])

I hate it.
I hate cats.
I hate politics.
I hate that.
I hate Trump.
I hate this.
I hate hate.
I hate war.
I hate change.
I hate math.
I hate America.
I hate feminism.
I hate technology.
I hate voting.
I hate pizza.
I hate science.
I hate government.
I hate Twitter.
I hate myself.
I hate religion.
I hate Obamacare.
I hate divorce.
I hate discrimination.
I hate violence.
I hate liberals.
I hate money.
I hate immigrants.
I hate Facebook.
I hate politicians.
I hate cancer.
I hate crowds.
I hate Obama.
I hate taxes.
I hate fascism.
I hate kids.
I hate food.
I hate everything.
I hate bureaucracy.
I hate terrorism.
I hate writing.
I hate life.
I hate democracy.
I hate cops.
I hate tattoos.
I hate Christmas.
I hate lying.
I hate school.
I hate confrontation.
I hate computers.
I hate IT.
I hate Brexit.
I hate Republicans.
I hate hospitals.
I hate losing.
I hate ghosts.
I hate beer.
I hate bees.
I hate teachers.
I hate censorship.
I hate work.
I hate coffee.
I hate Starbucks.
I hate prisons.
I h

In [40]:
count

338

In [41]:
len(predicts)

658